In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn


In [82]:
df = pd.read_csv("../raw data/online retail.csv" ,encoding='ISO-8859-1', header=0)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [83]:
df.shape

(541909, 8)

In [84]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [85]:
df=df.dropna()

In [86]:
df.shape

(406829, 8)

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [89]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


# Data Prepration

##### removes the space if present in the description
#####  removes the duplicate from the invoiceNo
##### Invoice: Requests payment from the buyer.
##### removes the  Credit Invoice: Reduces the amount owed, usually due to a correction or return.
##### change the data types of the invoice from int to string

In [90]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'],inplace=True)
# df['InvoiceNo'] =  df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains("C")] #remove the Credit invoice
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397924 non-null  object 
 1   StockCode    397924 non-null  object 
 2   Description  397924 non-null  object 
 3   Quantity     397924 non-null  int64  
 4   InvoiceDate  397924 non-null  object 
 5   UnitPrice    397924 non-null  float64
 6   CustomerID   397924 non-null  float64
 7   Country      397924 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


In [92]:
df['Country'].value_counts()

Country
United Kingdom          354345
Germany                   9042
France                    8342
EIRE                      7238
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1842
Portugal                  1462
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon                     45


In [93]:
df.shape

(397924, 8)

### Seperating transaction for Germany

In [94]:
mybasket = (
    df[df['Country'] == 'Germany']
    .groupby(['InvoiceNo', 'Description'])['Quantity']
    .sum()  # Use 'sum()' instead of 'Sum()'
    .unstack()  # Pivot the table
    .reset_index()  # Reset the index to make 'InvoiceNo' a column
    .fillna(0)  # Fill NaN values with 0
    .set_index('InvoiceNo')  # Set 'InvoiceNo' as the index
)


In [95]:
mybasket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
mybasket.to_csv('../Filtered data/mybasket_germany.csv')


In [97]:
def my_encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
my_basket_sets = mybasket.applymap(my_encode_units)
my_basket_sets.drop('POSTAGE', inplace=True, axis =1) #remove "postage" as an item

C:\Users\adity\AppData\Local\Temp\ipykernel_49100\783131121.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets = mybasket.applymap(my_encode_units)


# Training Model

In [98]:
# pip install mlxtend


In [99]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [100]:
#Generating frequent itemsets
my_frequent_itemsets = apriori(my_basket_sets,min_support=0.07, use_colnames=True)


C:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [101]:
#generating rules
my_rules = association_rules(my_frequent_itemsets,metric="lift",min_threshold=1)

In [102]:
my_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.137856,0.245077,0.074398,0.539683,2.202098,0.040613,1.640006,0.633174
1,(ROUND SNACK BOXES SET OF4 WOODLAND),(PLASTERS IN TIN WOODLAND ANIMALS),0.245077,0.137856,0.074398,0.303571,2.202098,0.040613,1.237951,0.723103
2,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540,0.837922
3,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509,0.935072
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(SPACEBOY LUNCH BOX),0.245077,0.102845,0.070022,0.285714,2.778116,0.044817,1.256018,0.847826


In [103]:
my_rules.shape

(6, 10)

# Making Recomendation

In [104]:
my_basket_sets['ROUND SNACK BOXES SET OF4 WOODLAND'].sum()


112

In [105]:
my_basket_sets['SPACEBOY LUNCH BOX'].sum()

47

In [106]:
my_basket_sets['ROUND SNACK BOXES SET OF 4 FRUITS'].sum()

72

In [107]:
my_basket_sets['ROUND SNACK BOXES SET OF4 WOODLAND'].sum()

112

In [108]:
# filtering rulese based on condition
my_rules[(my_rules['lift'] >= 2) &
        (my_rules['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.137856,0.245077,0.074398,0.539683,2.202098,0.040613,1.640006,0.633174
2,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540,0.837922
3,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509,0.935072
5,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,0.044817,2.365427,0.713415


In [109]:
from mlxtend.frequent_patterns import apriori, association_rules

# Generate frequent itemsets with very low support
frequent_itemsets = apriori(my_basket_sets, min_support=0.01, use_colnames=True)

# Generate association rules with very low confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# Display all rules
rules.head(2)



C:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(10 COLOUR SPACEBOY PEN),(LUNCH BAG APPLE DESIGN),0.024070,0.061269,0.010941,0.454545,7.418831,0.009466,1.721007,0.886547
1,(LUNCH BAG APPLE DESIGN),(10 COLOUR SPACEBOY PEN),0.061269,0.024070,0.010941,0.178571,7.418831,0.009466,1.188089,0.921678


In [110]:
my_basket_sets['ROUND SNACK BOXES SET OF4 WOODLAND'].sum()

112

In [111]:
my_basket_sets['SPACEBOY LUNCH BOX'].sum()

47

In [112]:
# Loop through the rules and print associations
for index, row in rules.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    print(f"{antecedents} -> {consequents}")

10 COLOUR SPACEBOY PEN -> LUNCH BAG APPLE DESIGN
LUNCH BAG APPLE DESIGN -> 10 COLOUR SPACEBOY PEN
10 COLOUR SPACEBOY PEN -> SPACEBOY LUNCH BOX
SPACEBOY LUNCH BOX -> 10 COLOUR SPACEBOY PEN
12 PENCILS TALL TUBE WOODLAND -> ROUND SNACK BOXES SET OF4 WOODLAND
ROUND SNACK BOXES SET OF4 WOODLAND -> 12 PENCILS TALL TUBE WOODLAND
3 HOOK HANGER MAGIC GARDEN -> 5 HOOK HANGER RED MAGIC TOADSTOOL
5 HOOK HANGER RED MAGIC TOADSTOOL -> 3 HOOK HANGER MAGIC GARDEN
3 HOOK HANGER MAGIC GARDEN -> JUMBO BAG APPLES
JUMBO BAG APPLES -> 3 HOOK HANGER MAGIC GARDEN
ROUND SNACK BOXES SET OF 4 FRUITS -> 3 HOOK HANGER MAGIC GARDEN
3 HOOK HANGER MAGIC GARDEN -> ROUND SNACK BOXES SET OF 4 FRUITS
3 HOOK HANGER MAGIC GARDEN -> ROUND SNACK BOXES SET OF4 WOODLAND
ROUND SNACK BOXES SET OF4 WOODLAND -> 3 HOOK HANGER MAGIC GARDEN
3 HOOK HANGER MAGIC GARDEN -> WOODLAND CHARLOTTE BAG
WOODLAND CHARLOTTE BAG -> 3 HOOK HANGER MAGIC GARDEN
PLASTERS IN TIN CIRCUS PARADE -> 3 PIECE SPACEBOY COOKIE CUTTER SET
3 PIECE SPACEBOY COOKI

In [113]:
from mlxtend.frequent_patterns import apriori, association_rules

# Generate frequent itemsets with very low support
frequent_itemsets = apriori(my_basket_sets, min_support=0.01, use_colnames=True)

# Generate association rules with very low confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# Function to find associations for a specific item
def find_associations(item):
    # Filter rules where the item is in the antecedents
    antecedent_rules = rules[rules['antecedents'].apply(lambda x: item in x)]
    # Filter rules where the item is in the consequents
    consequent_rules = rules[rules['consequents'].apply(lambda x: item in x)]
    
    print(f"Associations for item: {item}\n")
    
    # Print antecedent rules
    print("As Antecedent:")
    for index, row in antecedent_rules.iterrows():
        antecedents = ', '.join(list(row['antecedents']))
        consequents = ', '.join(list(row['consequents']))
        print(f"{antecedents} -> {consequents} (Confidence: {row['confidence']:.2f}, Lift: {row['lift']:.2f})")
    
    # Print consequent rules
    print("\nAs Consequent:")
    for index, row in consequent_rules.iterrows():
        antecedents = ', '.join(list(row['antecedents']))
        consequents = ', '.join(list(row['consequents']))
        print(f"{antecedents} -> {consequents} (Confidence: {row['confidence']:.2f}, Lift: {row['lift']:.2f})")

# Example usage
find_associations('LUNCH BAG APPLE DESIGN')


C:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Associations for item: LUNCH BAG APPLE DESIGN

As Antecedent:
LUNCH BAG APPLE DESIGN -> 10 COLOUR SPACEBOY PEN (Confidence: 0.18, Lift: 7.42)
LUNCH BAG APPLE DESIGN -> 6 RIBBONS RUSTIC CHARM (Confidence: 0.25, Lift: 2.43)
LUNCH BAG APPLE DESIGN -> ASSORTED COLOUR MINI CASES (Confidence: 0.18, Lift: 5.44)
LUNCH BAG APPLE DESIGN -> CARD CIRCUS PARADE (Confidence: 0.21, Lift: 5.76)
LUNCH BAG APPLE DESIGN -> CHARLOTTE BAG APPLES DESIGN (Confidence: 0.43, Lift: 6.53)
LUNCH BAG APPLE DESIGN -> CHARLOTTE BAG SUKI DESIGN (Confidence: 0.21, Lift: 4.66)
LUNCH BAG APPLE DESIGN -> CHARLOTTE BAG VINTAGE ALPHABET (Confidence: 0.18, Lift: 5.83)
LUNCH BAG APPLE DESIGN -> CIRCUS PARADE LUNCH BOX (Confidence: 0.18, Lift: 4.80)
LUNCH BAG APPLE DESIGN -> JUMBO BAG APPLES (Confidence: 0.39, Lift: 6.41)
LUNCH BAG APPLE DESIGN -> JUMBO BAG PEARS (Confidence: 0.18, Lift: 6.28)
LUNCH BAG APPLE DESIGN -> JUMBO BAG WOODLAND ANIMALS (Confidence: 0.21, Lift: 2.13)
LUNCH BAG APPLE DESIGN -> LUNCH BAG ALPHABET DESIG

In [114]:
# Check the columns in the rules DataFrame
print(rules.columns)

# Specify the item you are interested in
item = 'LUNCH BAG APPLE DESIGN'

# Filter rules where the item is in the antecedents
antecedent_rules = rules[rules['antecedents'].apply(lambda x: item in x)]
# Filter rules where the item is in the consequents
consequent_rules = rules[rules['consequents'].apply(lambda x: item in x)]

# Combine the results
associations = pd.concat([antecedent_rules, consequent_rules])

# Format the results
associations['Antecedents'] = associations['antecedents'].apply(lambda x: ', '.join(list(x)))
associations['Consequents'] = associations['consequents'].apply(lambda x: ', '.join(list(x)))

# Display the associations
print(associations[['antecedents', 'consequents', 'confidence', 'lift']])

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'zhangs_metric'],
      dtype='object')
                                             antecedents  \
1                               (LUNCH BAG APPLE DESIGN)   
94                              (LUNCH BAG APPLE DESIGN)   
289                             (LUNCH BAG APPLE DESIGN)   
548                             (LUNCH BAG APPLE DESIGN)   
605                             (LUNCH BAG APPLE DESIGN)   
...                                                  ...   
14249       (SPACEBOY BIRTHDAY CARD, SPACEBOY LUNCH BOX)   
14251  (ROUND SNACK BOXES SET OF4 WOODLAND, SPACEBOY ...   
14254                               (SPACEBOY LUNCH BOX)   
14255               (ROUND SNACK BOXES SET OF4 WOODLAND)   
14256                           (SPACEBOY BIRTHDAY CARD)   

                                             consequents  confidence  \
1            

In [115]:
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# Generate frequent itemsets with very low support
frequent_itemsets = apriori(my_basket_sets, min_support=0.01, use_colnames=True)

# Generate association rules with very low confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# Create lists to store the associations
antecedents_list = []
consequents_list = []
confidence_list = []
lift_list = []

# Loop through the rules and store associations
for index, row in rules.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    antecedents_list.append(antecedents)
    consequents_list.append(consequents)
    confidence_list.append(row['confidence'])
    lift_list.append(row['lift'])

# Create a DataFrame to store the associations
associations_df = pd.DataFrame({
    'Antecedents': antecedents_list,
    'Consequents': consequents_list,
    'Confidence': confidence_list,
    'Lift': lift_list
})

# Save the DataFrame to a CSV file
associations_df.to_csv('../Filtered data/associations.csv', index=False)

print("Associations saved to associations.csv")


C:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Associations saved to associations.csv


In [116]:
# Define thresholds for classification
high_confidence_threshold = 0.7
medium_confidence_threshold = 0.4
high_lift_threshold = 1.5
medium_lift_threshold = 1.0

# Create lists to store the associations
antecedents_list = []
consequents_list = []
confidence_list = []
lift_list = []
classification_list = []

# Loop through the rules and classify associations
for index, row in rules.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    confidence = row['confidence']
    lift = row['lift']
    
    # Classify based on confidence and lift
    if confidence >= high_confidence_threshold and lift >= high_lift_threshold:
        classification = 'Best'
    elif confidence >= medium_confidence_threshold and lift >= medium_lift_threshold:
        classification = 'Medium'
    else:
        classification = 'Low'
    
    # Store the values
    antecedents_list.append(antecedents)
    consequents_list.append(consequents)
    confidence_list.append(confidence)
    lift_list.append(lift)
    classification_list.append(classification)

# Create a DataFrame to store the associations
associations_df = pd.DataFrame({
    'Antecedents': antecedents_list,
    'Consequents': consequents_list,
    'Confidence': confidence_list,
    'Lift': lift_list,
    'Classification': classification_list
})

# Sort the DataFrame by classification
classification_order = {'Best': 1, 'Medium': 2, 'Low': 3}
associations_df['ClassificationOrder'] = associations_df['Classification'].map(classification_order)
associations_df = associations_df.sort_values(by='ClassificationOrder')

# Drop the helper column
associations_df = associations_df.drop(columns=['ClassificationOrder'])

# Save the DataFrame to a CSV file
associations_df.to_csv("../Filtered data/classified_associations_sorted.csv", index=False)

print("Classified associations saved to classified_associations.csv")

Classified associations saved to classified_associations.csv


In [117]:
def classify_and_save_associations(rules, high_confidence_threshold=0.7, medium_confidence_threshold=0.4, high_lift_threshold=1.5, medium_lift_threshold=1.0):
    # Create lists to store the associations
    best_antecedents = []
    best_consequents = []
    medium_antecedents = []
    medium_consequents = []
    low_antecedents = []
    low_consequents = []
    best_confidence = []
    best_lift = []
    medium_confidence = []
    medium_lift = []
    low_confidence = []
    low_lift = []

    # Loop through the rules and classify associations
    for index, row in rules.iterrows():
        antecedents = ', '.join(list(row['antecedents']))
        consequents = ', '.join(list(row['consequents']))
        confidence = row['confidence']
        lift = row['lift']
        
        # Classify based on confidence and lift
        if confidence >= high_confidence_threshold and lift >= high_lift_threshold:
            best_antecedents.append(antecedents)
            best_consequents.append(consequents)
            best_confidence.append(confidence)
            best_lift.append(lift)
        elif confidence >= medium_confidence_threshold and lift >= medium_lift_threshold:
            medium_antecedents.append(antecedents)
            medium_consequents.append(consequents)
            medium_confidence.append(confidence)
            medium_lift.append(lift)
        else:
            low_antecedents.append(antecedents)
            low_consequents.append(consequents)
            low_confidence.append(confidence)
            low_lift.append(lift)

    # Create DataFrames for each classification
    best_df = pd.DataFrame({
        'Antecedents': best_antecedents,
        'Consequents': best_consequents,
        'Confidence': best_confidence,
        'Lift': best_lift
    })

    medium_df = pd.DataFrame({
        'Antecedents': medium_antecedents,
        'Consequents': medium_consequents,
        'Confidence': medium_confidence,
        'Lift': medium_lift
    })

    low_df = pd.DataFrame({
        'Antecedents': low_antecedents,
        'Consequents': low_consequents,
        'Confidence': low_confidence,
        'Lift': low_lift
    })

    # Save each DataFrame to a separate CSV file
    best_df.to_csv('../Filtered data/best_associations.csv', index=False)
    medium_df.to_csv('../Filtered data/medium_associations.csv', index=False)
    low_df.to_csv('../Filtered data/low_associations.csv', index=False)

    print("Classified associations saved to separate CSV files.")

# Example usage
classify_and_save_associations(rules)


Classified associations saved to separate CSV files.


In [118]:
# Define thresholds for classification
high_confidence_threshold = 0.7
medium_confidence_threshold = 0.4
high_lift_threshold = 1.5
medium_lift_threshold = 1.0

# Create lists to store the associations
best_antecedents = []
best_consequents = []
medium_antecedents = []
medium_consequents = []
low_antecedents = []
low_consequents = []
best_confidence = []
best_lift = []
medium_confidence = []
medium_lift = []
low_confidence = []
low_lift = []

# Loop through the rules and classify associations
for index, row in rules.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    confidence = row['confidence']
    lift = row['lift']
    
    # Classify based on confidence and lift
    if confidence >= high_confidence_threshold and lift >= high_lift_threshold:
        best_antecedents.append(antecedents)
        best_consequents.append(consequents)
        best_confidence.append(confidence)
        best_lift.append(lift)
    elif confidence >= medium_confidence_threshold and lift >= medium_lift_threshold:
        medium_antecedents.append(antecedents)
        medium_consequents.append(consequents)
        medium_confidence.append(confidence)
        medium_lift.append(lift)
    else:
        low_antecedents.append(antecedents)
        low_consequents.append(consequents)
        low_confidence.append(confidence)
        low_lift.append(lift)

# Create DataFrames for each classification
best_df = pd.DataFrame({
    'Antecedents': best_antecedents,
    'Consequents': best_consequents,
    'Confidence': best_confidence,
    'Lift': best_lift
})

medium_df = pd.DataFrame({
    'Antecedents': medium_antecedents,
    'Consequents': medium_consequents,
    'Confidence': medium_confidence,
    'Lift': medium_lift
})

low_df = pd.DataFrame({
    'Antecedents': low_antecedents,
    'Consequents': low_consequents,
    'Confidence': low_confidence,
    'Lift': low_lift
})

In [119]:
import pickle
# Save each DataFrame to a separate pickle file
with open('../model pkl files/best_associations.pkl', 'wb') as f:
    pickle.dump(best_df, f)

with open('../model pkl files/medium_associations.pkl', 'wb') as f:
    pickle.dump(medium_df, f)

with open('../model pkl files/low_associations.pkl', 'wb') as f:
    pickle.dump(low_df, f)

print("Classified associations saved to separate pickle files.")

Classified associations saved to separate pickle files.


#### Now the last code is for the only when you want the best associated and best pkl files then u don't use above one's directly run the last one 

In [120]:
# # Define thresholds for classification
# high_confidence_threshold = 0.7
# medium_confidence_threshold = 0.4
# high_lift_threshold = 1.5
# medium_lift_threshold = 1.0

# # Create lists to store the associations
# antecedents_list = []
# consequents_list = []
# confidence_list = []
# lift_list = []
# classification_list = []

# # Loop through the rules and classify associations
# for index, row in rules.iterrows():
#     antecedents = ', '.join(list(row['antecedents']))
#     consequents = ', '.join(list(row['consequents']))
#     confidence = row['confidence']
#     lift = row['lift']
    
#     # Classify based on confidence and lift
#     if confidence >= high_confidence_threshold and lift >= high_lift_threshold:
#         classification = 'Best'
#     elif confidence >= medium_confidence_threshold and lift >= medium_lift_threshold:
#         classification = 'Medium'
#     else:
#         classification = 'Low'
    
#     # Store the values
#     antecedents_list.append(antecedents)
#     consequents_list.append(consequents)
#     confidence_list.append(confidence)
#     lift_list.append(lift)
#     classification_list.append(classification)

# # Create a DataFrame to store the associations
# associations_df = pd.DataFrame({
#     'Antecedents': antecedents_list,
#     'Consequents': consequents_list,
#     'Confidence': confidence_list,
#     'Lift': lift_list,
#     'Classification': classification_list
# })

# # Sort the DataFrame by classification
# classification_order = {'Best': 1, 'Medium': 2, 'Low': 3}
# associations_df['ClassificationOrder'] = associations_df['Classification'].map(classification_order)
# associations_df = associations_df.sort_values(by='ClassificationOrder')

# # Drop the helper column
# associations_df = associations_df.drop(columns=['ClassificationOrder'])

# # Save the DataFrame to a CSV file
# associations_df.to_csv('../Filtered data/classified_associations_sorted.csv', index=False)

# print("Classified associations saved to classified_associations.csv")

# # Save each DataFrame to a separate pickle file
# with open('../model pkl files/best_associations.pkl', 'wb') as f:
#     pickle.dump(associations_df, f)